# ETL with EEOC
# Khrizel Solano and Kelli Okuji Wilson

# Extract 

In [603]:
#pip install mysqlclient
import fnmatch
from regexdict import regexdict
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from pandas.io import sql
import MySQLdb

In [604]:
# Store CSV into DataFrame

In [605]:
csv_file = "./Resources/year16_nac2.csv"
eeoc_nac2_df = pd.read_csv(csv_file, sep=';').dropna(how='all')
eeoc_nac2_df.sample(5)
#eeoc_db.year16_state_nac2.csv

,NAC2_code,TOTAL_UNITS,TOTAL10,WHM1,WHM2,WHM3,WHM4,WHM5,WHM6,WHM7,...,TOMRF7,TOMRF8,TOMRF9,TOMRF1_2,NAC2_Label,i,SUMCOUNT,MISSCOUNT,SMALLEST,INDEX
12,51,8407,1731043,21470,271515,64311,63472,64590,52241,13782,...,213,110.0,1456,1572,Information,141,0,0,NaN,NaN
19,62,39106,10506848,47256,513724,208814,7219,74277,49228,27293,...,530,2868.0,40199,4803,Health Care and Social Assistance,141,0,0,NaN,NaN
0,11,1408,321995,3311,7254,2369,4740,2561,5636,12178,...,68,871.0,35,43,"Agriculture, Forestry, Fishing and Hunting",141,4,2,3.0,84.0
21,72,28756,3304591,14763,16435,4661,13407,17664,18548,14340,...,343,1304.0,44330,2404,Accommodation and Food Services,141,0,0,NaN,NaN
16,55,4583,770594,20750,94025,13700,19372,20457,13859,15432,...,88,244.0,802,776,Management of Companies and Enterprises,141,0,0,NaN,NaN


In [606]:
eeoc_kaggle_csv_file = "./Resources/Reveal_EEO1_for_2016.csv"
eeoc_kaggle_df = pd.read_csv(eeoc_kaggle_csv_file).dropna(how='all')
eeoc_kaggle_df.sample(10)
#Reveal_EEO1_for_2016.csv

,company,year,race,gender,job_category,count
844,Cisco,2016,Native_Hawaiian_or_Pacific_Islander,female,Sales workers,3
3736,Uber,2016,Asian,female,Sales workers,2
3869,View,2016,Two_or_more_races,male,Administrative support,0
1041,eBay,2016,Asian,female,Service workers,0
3610,Uber,2016,Latino,male,Totals,174
2943,Pinterest,2016,Asian,male,Technicians,0
529,Airbnb,2016,Overall_totals,NaN,First/Mid officials & Mgrs,295
3957,View,2016,Overall_totals,NaN,Service workers,0
1078,eBay,2016,Overall_totals,NaN,Totals,6611
2318,LinkedIn,2016,Two_or_more_races,female,Professionals,36


## Transform

In [607]:
# Create new data sets with select columns

In [608]:
eeoc_nac2_All_df = eeoc_nac2_df[['NAC2_code','NAC2_Label','WHM1','WHM2','WHM3','WHM4','WHM5','WHM6','WHM7','WHM8','WHM9','WHF1','WHF2','WHF3','WHF4','WHF5','WHF6','WHF7','WHF8','WHF9','BLKM1','BLKM2','BLKM3','BLKM4','BLKM5','BLKM6','BLKM7','BLKM8','BLKM9','BLKF1','BLKF2','BLKF3','BLKF4','BLKF5','BLKF6','BLKF7','BLKF8','BLKF9','HISPM1','HISPM2','HISPM3','HISPM4','HISPM5','HISPM6','HISPM7','HISPM8','HISPM9','HISPF1','HISPF2','HISPF3','HISPF4','HISPF5','HISPF6','HISPF7','HISPF8','HISPF9','ASIANM1','ASIANM2','ASIANM3','ASIANM4','ASIANM5','ASIANM6','ASIANM7','ASIANM8','ASIANM9','ASIANF1','ASIANF2','ASIANF3','ASIANF4','ASIANF5','ASIANF6','ASIANF7','ASIANF8','ASIANF9','AIANM1','AIANM2','AIANM3','AIANM4','AIANM5','AIANM6','AIANM7','AIANM8','AIANM9','AIANF1','AIANF2','AIANF3','AIANF4','AIANF5','AIANF6','AIANF7','AIANF8','AIANF9','NHOPIM1','NHOPIM2','NHOPIM3','NHOPIM4','NHOPIM5','NHOPIM6','NHOPIM7','NHOPIM8','NHOPIM9','NHOPIF1','NHOPIF2','NHOPIF3','NHOPIF4','NHOPIF5','NHOPIF6','NHOPIF7','NHOPIF8','NHOPIF9','TOMRM1','TOMRM2','TOMRM3','TOMRM4','TOMRM5','TOMRM6','TOMRM7','TOMRM8','TOMRM9','TOMRF1','TOMRF2','TOMRF3','TOMRF4','TOMRF5','TOMRF6','TOMRF7','TOMRF8','TOMRF9']]
eeoc_nac2_All_df.head()

,NAC2_code,NAC2_Label,WHM1,WHM2,WHM3,WHM4,WHM5,WHM6,WHM7,WHM8,...,TOMRM9,TOMRF1,TOMRF2,TOMRF3,TOMRF4,TOMRF5,TOMRF6,TOMRF7,TOMRF8,TOMRF9
0,11,"Agriculture, Forestry, Fishing and Hunting",3311,7254,2369,4740,2561,5636,12178,15711,...,71,6,58,26,35.0,100,5,68,871.0,35
1,21,"Mining, Quarrying, and Oil and Gas Extraction",5039,37459,14647,2088,3999,58589,43730,15476,...,12,6,322,71,10.0,214,13,32,11.0,8
2,22,Utilities,5883,64738,25510,1645,9568,87506,21627,6961,...,45,13,600,63,22.0,682,28,20,9.0,4
3,23,Construction,26632,83888,37250,21458,24350,393797,91062,114587,...,173,43,529,101,251.0,1436,274,88,196.0,52
4,31,Manufacturing,15623,39393,18702,39607,17628,83772,174938,80811,...,731,37,621,213,462.0,816,128,1300,1630.0,755


In [609]:
# Convert wide to narrow table using melt

In [610]:
eeoc_nac2_All_unpivot_df=pd.melt(eeoc_nac2_All_df, id_vars=['NAC2_code','NAC2_Label'], var_name="EEOC_Code", value_name="count").dropna(how='all')
eeoc_nac2_All_unpivot_df.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count
732,51,Information,BLKF4,7980.0
1431,54,"Professional, Scientific, and Technical Services",ASIANM6,1476.0
1934,53,Real Estate and Rental and Leasing,AIANM9,127.0
1447,42,Wholesale Trade,ASIANM7,7574.0
1050,61,Educational Services,HISPM8,1512.0


In [611]:
#Transform EEOC data

In [612]:
eeoc_nac2_All_unpivot_df['gender'] = eeoc_nac2_All_unpivot_df['EEOC_Code'].str[-2].replace({'M' : 'Male', 'F' : 'Female'})
eeoc_nac2_All_unpivot_df.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count,gender
808,55,Management of Companies and Enterprises,BLKF7,1454.0,Female
439,42,Wholesale Trade,BLKM1,348.0,Male
1762,48,Transportation and Warehousing,AIANM2,816.0,Male
1984,55,Management of Companies and Enterprises,AIANF2,572.0,Female
2639,92,Public Administration,TOMRM2,247.0,Male


In [613]:
eeoc_nac2_All_unpivot_df['job_category'] = eeoc_nac2_All_unpivot_df['EEOC_Code'].str[-1].replace({'1' : 'Senior OFF AND MGRS', '2' : 'PROF', '3' : 'TECH', '4' : 'SALE', '5' : 'CLERICALS', '6' : 'CRAFT', '7' : 'OPERS','8' : 'LABORS', '9' : 'Service'}).dropna(how='all')
eeoc_nac2_All_unpivot_df.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count,gender,job_category
2580,51,Information,NHOPIF9,180.0,Female,Service
713,56,Administrative and Support and Waste Managemen...,BLKF3,7403.0,Female,TECH
461,32,Manufacturing,BLKM2,8917.0,Male,PROF
1524,51,Information,ASIANF1,739.0,Female,Senior OFF AND MGRS
2546,22,Utilities,NHOPIF8,3.0,Female,LABORS


In [614]:
eeoc_nac2_All_unpivot_df['race'] = eeoc_nac2_All_unpivot_df['EEOC_Code'].str[:-2].replace({'WH' : 'White', 'BLK' : 'Black', 'HISP': 'Hispanic_or_Latino','ASIAN' : 'Asian','AIAN' : 'American_Indian_Alaskan_Native','NHOPI' : 'Native_Hawaiian_or_Pacific_Islander','TOMR' : 'Two_or_more_races'}).dropna(how='all')
eeoc_nac2_All_unpivot_df.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count,gender,job_category,race
287,92,Public Administration,WHF3,424.0,Female,TECH,White
2822,53,Real Estate and Rental and Leasing,TOMRF1,65.0,Female,Senior OFF AND MGRS,Two_or_more_races
1419,23,Construction,ASIANM6,7833.0,Male,CRAFT,Asian
318,33,Manufacturing,WHF5,162935.0,Female,CLERICALS,White
3002,22,Utilities,TOMRF9,4.0,Female,Service,Two_or_more_races


In [615]:
#Transform Kaggle data

In [616]:
eeoc_kaggle_df.sample(5)

,company,year,race,gender,job_category,count
1229,Facebook,2016,American_Indian_Alaskan_Native,female,Administrative support,2
1826,Intel,2016,White,male,Professionals,12352
1406,Google,2016,American_Indian_Alaskan_Native,female,Professionals,7
747,Cisco,2016,White,male,Technicians,11
3530,Twitter,2016,Black_or_African_American,female,Professionals,18


In [617]:
eeoc_kaggle_df['gender'] = eeoc_kaggle_df['gender'].replace({'male' : 'Male', 'female' : 'Female'}).dropna(how='all')
eeoc_kaggle_df.sample(5)

,company,year,race,gender,job_category,count
870,Cisco,2016,American_Indian_Alaskan_Native,Female,Craft workers,0
701,Apple,2016,Two_or_more_races,Female,Administrative support,94
3788,View,2016,Latino,Male,laborers and helpers,0
1130,Facebook,2016,Native_Hawaiian_or_Pacific_Islander,Male,Professionals,9
1774,HPE,2016,American_Indian_Alaskan_Native,Female,Totals,86


In [618]:
eeoc_kaggle_df['job_category'] = eeoc_kaggle_df['job_category'].replace({'Executive/Senior officials & Mgrs' : 'Senior OFF AND MGRS', 'First/Mid officials & Mgrs' : 'Senior OFF AND MGRS', 'Professionals' : 'PROF', 'Technicians' : 'TECH', 'Sales workers' : 'SALE', 'Administrative support' : 'CLERICALS', 'Craft workers' : 'CRAFT', 'operatives' : 'OPERS','laborers and helpers' : 'LABORS', 'Service workers' : 'Service'}).dropna(how=all)
eeoc_kaggle_df.sample(5)

,company,year,race,gender,job_category,count
1412,Google,2016,American_Indian_Alaskan_Native,Female,LABORS,0
1016,eBay,2016,Black_or_African_American,Female,LABORS,0
439,Airbnb,2016,American_Indian_Alaskan_Native,Male,OPERS,0
2278,LinkedIn,2016,Black_or_African_American,Female,Totals,70
1851,Intel,2016,Native_Hawaiian_or_Pacific_Islander,Male,TECH,37


In [619]:
eeoc_kaggle_df_clean = eeoc_kaggle_df[(eeoc_kaggle_df["job_category"]!= "Totals") & (eeoc_kaggle_df["job_category"]!= "Previous_totals") & (eeoc_kaggle_df["job_category"]!= "Overall_totals")&(eeoc_kaggle_df["race"]!= "Previous_totals") & (eeoc_kaggle_df["race"]!= "Overall_totals")]
eeoc_kaggle_df_clean.sample(5)

,company,year,race,gender,job_category,count
3741,Uber,2016,Asian,Female,Service,0
2540,MobileIron,2016,Latino,Female,LABORS,0
2972,Pinterest,2016,Two_or_more_races,Male,LABORS,0
67,23andMe,2016,Asian,Male,OPERS,0
2680,MobileIron,2016,Two_or_more_races,Female,SALE,0


In [620]:
#join these 2 data sets: EEOC and Kaggle

In [621]:
eeoc_nac2_All_unpivot_df.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count,gender,job_category,race
259,62,Health Care and Social Assistance,WHF2,2177598.0,Female,PROF,White
11,49,Transportation and Warehousing,WHM1,2697.0,Male,Senior OFF AND MGRS,White
2807,92,Public Administration,TOMRM9,70.0,Male,Service,Two_or_more_races
425,56,Administrative and Support and Waste Managemen...,WHF9,99600.0,Female,Service,White
142,81,Other Services (except Public Administration),WHM6,26124.0,Male,CRAFT,White


In [622]:
eeoc_kaggle_df_clean.sample(5)

,company,year,race,gender,job_category,count
3247,Square,2016,Latino,Male,OPERS,0
1916,Intel,2016,Black_or_African_American,Female,LABORS,0
2817,Nvidia,2016,Black_or_African_American,Female,Service,0
474,Airbnb,2016,Black_or_African_American,Female,CRAFT,0
2994,Pinterest,2016,Black_or_African_American,Female,CRAFT,0


In [623]:
kaggle_female=eeoc_kaggle_df_clean[(eeoc_kaggle_df_clean['gender']=='Female')]
kaggle_female.sample(5)

,company,year,race,gender,job_category,count
3012,Pinterest,2016,Asian,Female,Senior OFF AND MGRS,0
1910,Intel,2016,Black_or_African_American,Female,PROF,356
1219,Facebook,2016,Asian,Female,OPERS,0
481,Airbnb,2016,Native_Hawaiian_or_Pacific_Islander,Female,Senior OFF AND MGRS,0
509,Airbnb,2016,American_Indian_Alaskan_Native,Female,CLERICALS,1


In [624]:
eeoc_female=eeoc_nac2_All_unpivot_df[(eeoc_nac2_All_unpivot_df['NAC2_Label']  =='Professional, Scientific, and Technical Services') & (eeoc_nac2_All_unpivot_df['gender']  =='Female' )] 
eeoc_female.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count,gender,job_category,race
2583,54,"Professional, Scientific, and Technical Services",NHOPIF9,104.0,Female,Service,Native_Hawaiian_or_Pacific_Islander
2487,54,"Professional, Scientific, and Technical Services",NHOPIF5,1691.0,Female,CLERICALS,Native_Hawaiian_or_Pacific_Islander
1695,54,"Professional, Scientific, and Technical Services",ASIANF8,948.0,Female,LABORS,Asian
2079,54,"Professional, Scientific, and Technical Services",AIANF6,71.0,Female,CRAFT,American_Indian_Alaskan_Native
2127,54,"Professional, Scientific, and Technical Services",AIANF8,184.0,Female,LABORS,American_Indian_Alaskan_Native


In [625]:
eeoc_female_total=eeoc_female['count'].astype(int).sum()
eeoc_female_total

1340758

In [626]:
kaggle_female['CA count']=eeoc_female_total
kaggle_female.sample(5)

/Users/khrizel/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,company,year,race,gender,job_category,count,CA count
2647,MobileIron,2016,Native_Hawaiian_or_Pacific_Islander,Female,OPERS,0,1340758
689,Apple,2016,American_Indian_Alaskan_Native,Female,CLERICALS,17,1340758
459,Airbnb,2016,White,Female,TECH,3,1340758
2648,MobileIron,2016,Native_Hawaiian_or_Pacific_Islander,Female,LABORS,0,1340758
1772,HPE,2016,American_Indian_Alaskan_Native,Female,LABORS,0,1340758


In [627]:
eeoc_male=eeoc_nac2_All_unpivot_df[(eeoc_nac2_All_unpivot_df['NAC2_Label']  =='Professional, Scientific, and Technical Services') & (eeoc_nac2_All_unpivot_df['gender']  =='Male' )] 
eeoc_male.sample(5)

,NAC2_code,NAC2_Label,EEOC_Code,count,gender,job_category,race
1839,54,"Professional, Scientific, and Technical Services",AIANM5,576.0,Male,CLERICALS,American_Indian_Alaskan_Native
1911,54,"Professional, Scientific, and Technical Services",AIANM8,264.0,Male,LABORS,American_Indian_Alaskan_Native
183,54,"Professional, Scientific, and Technical Services",WHM8,15992.0,Male,LABORS,White
2703,54,"Professional, Scientific, and Technical Services",TOMRM5,4035.0,Male,CLERICALS,Two_or_more_races
1407,54,"Professional, Scientific, and Technical Services",ASIANM5,11072.0,Male,CLERICALS,Asian


In [628]:
eeoc_male=eeoc_nac2_All_unpivot_df[(eeoc_nac2_All_unpivot_df['NAC2_Label']  =='Professional, Scientific, and Technical Services') & (eeoc_nac2_All_unpivot_df['gender']  =='Male' )] 
eeoc_male_total=eeoc_male['count'].astype(int).sum()
eeoc_male_total

1803832

In [629]:
kaggle_male=eeoc_kaggle_df_clean[(eeoc_kaggle_df_clean['gender']=='Male')]
kaggle_male['CA count']=eeoc_male_total
kaggle_male.sample(5)

/Users/khrizel/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,company,year,race,gender,job_category,count,CA count
2762,Nvidia,2016,Asian,Male,PROF,1741,1803832
3603,Uber,2016,Latino,Male,TECH,2,1803832
1136,Facebook,2016,Native_Hawaiian_or_Pacific_Islander,Male,LABORS,0,1803832
2385,Lyft,2016,Black_or_African_American,Male,Service,1,1803832
1890,Intel,2016,Two_or_more_races,Male,CRAFT,6,1803832


In [630]:
eeoc_white=eeoc_nac2_All_unpivot_df[(eeoc_nac2_All_unpivot_df['NAC2_Label']  =='Professional, Scientific, and Technical Services') & (eeoc_nac2_All_unpivot_df['race']  =='White' )] 
eeoc_white_total=eeoc_white['count'].astype(int).sum()
eeoc_white_total

2087577

In [631]:
kaggle_white=eeoc_kaggle_df_clean[(eeoc_kaggle_df_clean['race']=='White')]
kaggle_white['CA count']=eeoc_white_total
kaggle_white.drop_duplicates()
kaggle_white.sample(5)

/Users/khrizel/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,company,year,race,gender,job_category,count,CA count
392,Airbnb,2016,White,Male,LABORS,0,2087577
3627,Uber,2016,White,Male,TECH,24,2087577
1285,Google,2016,White,Male,Senior OFF AND MGRS,4444,2087577
284,Adobe,2016,White,Female,LABORS,0,2087577
3518,Twitter,2016,White,Female,PROF,314,2087577


In [632]:
eeoc_manager=eeoc_nac2_All_unpivot_df[(eeoc_nac2_All_unpivot_df['NAC2_Label']  =='Professional, Scientific, and Technical Services') & (eeoc_nac2_All_unpivot_df['job_category']  =='Senior OFF AND MGRS' )] 
eeoc_manager_total=eeoc_manager['count'].astype(int).sum()
eeoc_manager_total

137156

In [633]:
kaggle_manager=eeoc_kaggle_df_clean[(eeoc_kaggle_df_clean['job_category']=='Senior OFF AND MGRS')]
kaggle_manager['CA count']=eeoc_manager_total
kaggle_manager.drop_duplicates()
kaggle_manager.sample(5)

/Users/khrizel/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,company,year,race,gender,job_category,count,CA count
2281,LinkedIn,2016,Native_Hawaiian_or_Pacific_Islander,Female,Senior OFF AND MGRS,2,137156
1284,Google,2016,White,Male,Senior OFF AND MGRS,20,137156
337,Adobe,2016,Two_or_more_races,Female,Senior OFF AND MGRS,7,137156
2029,Intuit,2016,Native_Hawaiian_or_Pacific_Islander,Male,Senior OFF AND MGRS,1,137156
1188,Facebook,2016,Black_or_African_American,Female,Senior OFF AND MGRS,4,137156


In [634]:
eeoc_prof=eeoc_nac2_All_unpivot_df[(eeoc_nac2_All_unpivot_df['NAC2_Label']  =='Professional, Scientific, and Technical Services') & (eeoc_nac2_All_unpivot_df['job_category']  =='PROF' )] 
eeoc_prof_total=eeoc_prof['count'].astype(int).sum()
eeoc_prof_total

1811785

In [635]:
kaggle_prof=eeoc_kaggle_df_clean[(eeoc_kaggle_df_clean['job_category']=='PROF')]
kaggle_prof['CA count']=eeoc_prof_total
kaggle_prof.drop_duplicates()
kaggle_prof.sample(5) 

/Users/khrizel/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,company,year,race,gender,job_category,count,CA count
2786,Nvidia,2016,Two_or_more_races,Male,PROF,14,1811785
2306,LinkedIn,2016,American_Indian_Alaskan_Native,Female,PROF,1,1811785
1802,Intel,2016,Latino,Male,PROF,1654,1811785
3854,View,2016,American_Indian_Alaskan_Native,Male,PROF,0,1811785
1274,Google,2016,Latino,Female,PROF,376,1811785


In [636]:
kaggle_white=eeoc_kaggle_df_clean[(eeoc_kaggle_df_clean['race']=='White')]
kaggle_white['CA count']=eeoc_white_total
kaggle_white.sample(5)

/Users/khrizel/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,company,year,race,gender,job_category,count,CA count
1723,HPE,2016,White,Female,OPERS,7,2087577
1284,Google,2016,White,Male,Senior OFF AND MGRS,20,2087577
207,Adobe,2016,White,Male,TECH,0,2087577
1899,Intel,2016,White,Female,TECH,680,2087577
2733,Nvidia,2016,White,Male,Service,0,2087577


In [637]:
eeoc_asian=eeoc_nac2_All_unpivot_df[(eeoc_nac2_All_unpivot_df['NAC2_Label']  =='Professional, Scientific, and Technical Services') & (eeoc_nac2_All_unpivot_df['race']  =='Asian' )] 
eeoc_asian_total=eeoc_asian['count'].astype(int).sum()
eeoc_asian_total

414202

In [638]:
kaggle_asian=eeoc_kaggle_df_clean[(eeoc_kaggle_df_clean['race']=='Asian')]
kaggle_asian['CA count']=eeoc_asian_total
kaggle_asian.drop_duplicates()
kaggle_asian.sample(5)

/Users/khrizel/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,company,year,race,gender,job_category,count,CA count
676,Apple,2016,Asian,Female,SALE,579,414202
2479,Lyft,2016,Asian,Female,OPERS,0,414202
606,Apple,2016,Asian,Male,CRAFT,12,414202
241,Adobe,2016,Asian,Male,Senior OFF AND MGRS,310,414202
1681,HPE,2016,Asian,Male,Senior OFF AND MGRS,442,414202


# Load

In [639]:
rds_connection_string = "root:mysqlp5ssw0rd*@127.0.0.1/eeoc_db" #hide this
engine = create_engine(f'mysql://{rds_connection_string}')

In [648]:
kaggle_female.to_sql(name='EEOC_Female', con=engine, if_exists='append', index=False)

In [641]:
kaggle_male.to_sql(name='EEOC_Male', con=engine, if_exists='append', index=False)

In [649]:
kaggle_manager.to_sql(name='EEOC_Manager', con=engine, if_exists='append', index=False)

In [643]:
kaggle_prof.to_sql(name='EEOC_Professional', con=engine, if_exists='append', index=False)

In [644]:
kaggle_white.to_sql(name='EEOC_White', con=engine, if_exists='append', index=False)

In [647]:
kaggle_asian.to_sql(name='EEOC_Asian', con=engine, if_exists='append', index=False)

## Summary

Extract 

We chose data sets for ETL from Kaggle and the EEOC website. Both data sets have the following fields in common: gender, race, and job category. The Kaggle data set is broken by company while the EEOC data set by state. Both files were in csv and were read in pandas to be used as data frames.

Transformation

* Transformed the EEOC table from wide to narrow so it's easier to join with the Kaggle data set
* Created EEOC data dictionaries for race, gender, and job category by parsing out the EEOC codes in EEOC data set to match the Kaggle data set values
* Cleaned up the values in the job category in the Kaggle data set so it matches with the EEOC data set values
* Droped nulls, duplicates, and total/subtotal lines in Kaggle data set
* Filtered out the EEOC and Kaggle dataframes by female, male, manager, professional, white, and asian 
* Sumed up the count for CA ’Professional, Scientific, and Technical Services' in the EEOC dataframes and create a new column to the corresponding Kaggle dataframest 

Load

* Loaded the EEOC female, male, manager, professional, white, and asian data frames as tables in MySQL

SQL Schema

* Created the eeoc_db database
* Created the female, male, manager, professional, white, and asian tables and assigned primary keys where applicable (there were duplicates after normalizing the 'Senior OFF AND MGRS' job category values from 'Executive/Senior officials & Mgrs' and 'First/Mid officials & Mgrs’)
* Joined Asian and White counts by gender and job category
* Joined Professional and Managers by race and gender